# 
Yolo8云端部署
#### 2025.4.14[中继训练]
## 过程说明:
1. 分别载入模型和数据集
2. 整体迁移至工程文件夹下
3. 运行训练脚本
4. [可选]运行推理，结果保存在/kaggle/working/pred_rst.jpg文件
5. 转存训练好的权重文件至/kaggle/working/train.zip
## 注意:
1. 开启Kaggle平台的GPU（P100）后，运行nvidia --smi 可以看到该GPU支持CUDA12.6版本。而且note环境里已经预装了CUDA11.2、torch2.5.1+cu121
2. 运行前检查：模型文件位置、数据集位置、数据集配置文件内容

In [1]:
# 工程准备
print("复制工程到根目录")
!cp -r /kaggle/input/yolo8/pytorch/1.0/17/yolo8/yolo8 /yolo8
!cp /kaggle/input/yolo8/pytorch/1.0/17/t300.pt /yolo8/best.pt
!ls /yolo8 -a
print("安装依赖")
!pip install -e /yolo8/
# 数据集准备
print("转移数据库")
!cp -r /kaggle/input/ocean3 /yolo8/datasets
#!cp -r /kaggle/input/river-floating-trash-datasets/datasets /yolo8/datasets
!ls /yolo8/datasets -a

# 测试已通过
print("准备完成")

复制工程到根目录
.		 docs	      noteB		       task
..		 examples     noteB.ipynb	       tests
best.pt		 .git	      .pre-commit-config.yaml  ultralytics
CITATION.cff	 .gitignore   README.md		       ultralytics.egg-info
CONTRIBUTING.md  LICENSE      README.zh-CN.md	       yolov8n.pt
datasets	 MANIFEST.in  requirements.txt
default.jpg	 mkdocs.yml   setup.cfg
docker		 none.txt     setup.py
安装依赖
Obtaining file:///yolo8
  Preparing metadata (setup.py) ... done
  Running setup.py develop for ultralytics
转移数据库
.  ..  oc_cld.yaml  ocean3  oc_loc.yaml
准备完成


In [2]:
cd /yolo8

/yolo8


In [3]:
ls ./ -a

./               docs/        noteB                    task/
../              examples/    noteB.ipynb              tests/
best.pt          .git/        .pre-commit-config.yaml  ultralytics/
CITATION.cff     .gitignore   README.md                ultralytics.egg-info/
CONTRIBUTING.md  LICENSE      README.zh-CN.md          yolov8n.pt
datasets/        MANIFEST.in  requirements.txt
default.jpg      mkdocs.yml   setup.cfg
docker/          none.txt     setup.py


In [4]:
# 开始推理

# 所有相对路径基于/yolo8


# 0训练    1推理
way = 0


def train():
    from ultralytics import YOLO
    # Load a model
    #model = YOLO("yolov8n.yaml")  # build a new model from scratch
    model = YOLO("best.pt")  # load a pretrained model (recommended for training)

    # Use the model
    results = model.train(data="./datasets/oc_cld.yaml", \
                          epochs=20,\
                          imgsz=640,\
                          batch=32,\
                          device=0)  # 0=GPU

def pred(image_path,rst_path):
    from ultralytics import YOLO
    import cv2
    import yaml

    # 加载训练好的权重
    model = YOLO('./runs/detect/train/weights/best.pt')
    results = model.predict(source=image_path)

    # 读取原始图像
    image = cv2.imread(image_path)
    cv2.imwrite('/kaggle/working/pred_src.jpg',image)

    # 处理推理结果
    for result in results:
        boxes = result.boxes  # 获取边界框信息
        if boxes is not None:
            for box in boxes:
                # 获取边界框坐标
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                x1, y1, x2, y2 = xyxy

                # 获取类别索引和置信度
                cls = int(box.cls[0])
                conf = float(box.conf[0])

                with open('./datasets/RFT.yaml', 'r') as f:
                    data = yaml.safe_load(f)
                names = data['names']

                # 绘制边界框
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # 准备标签文本
                label = f'{names[cls]}: {conf:.2f}'

                # 绘制标签背景
                (label_width, label_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                cv2.rectangle(image, (x1, y1 - label_height - 5), (x1 + label_width, y1), (0, 255, 0), -1)

                # 绘制标签文本
                cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    # 保存带有检测结果的图像
    return cv2.imwrite(rst_path, image)

if way==0:
    train()
# 训练结果转存
    !zip -r /kaggle/working/train.zip /yolo8/runs/detect/train
else:
    rstImg_path = '/kaggle/working/pred_rst.jpg'
    # 进行单张图像推理  6,15,19,22,31,35,40,43,46,72,73,78,80,85,88,92
    rst = pred(image_path='./datasets/RFT/images/test/000006.jpg',rst_path=rstImg_path)
    if rst:
        print(f"保存成功：{rstImg_path}")
    else:
        print("出错")

/yolo8/ultralytics/nn/tasks.py:336: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu')  # load
Ultralytics YOLOv8.0.29 🚀 Python-3.10.1

  adding: yolo8/runs/detect/train/ (stored 0%)
  adding: yolo8/runs/detect/train/val_batch1_pred.jpg (deflated 9%)
  adding: yolo8/runs/detect/train/events.out.tfevents.1744851628.e7a16e1335a7.18.0 (deflated 73%)
  adding: yolo8/runs/detect/train/train_batch2.jpg (deflated 14%)
  adding: yolo8/runs/detect/train/train_batch4340.jpg (deflated 14%)
  adding: yolo8/runs/detect/train/val_batch2_pred.jpg (deflated 15%)
  adding: yolo8/runs/detect/train/val_batch2_labels.jpg (deflated 15%)
  adding: yolo8/runs/detect/train/train_batch4342.jpg (deflated 10%)
  adding: yolo8/runs/detect/train/P_curve.png (deflated 10%)
  adding: yolo8/runs/detect/train/train_batch4341.jpg (deflated 13%)
  adding: yolo8/runs/detect/train/weights/ (stored 0%)
  adding: yolo8/runs/detect/train/weights/best.pt (deflated 9%)
  adding: yolo8/runs/detect/train/weights/last.pt (deflated 9%)
  adding: yolo8/runs/detect/train/F1_curve.png (deflated 12%)
  adding: yolo8/runs/detect/train/results.csv (deflated 84%)
  addin

In [5]:
ls -a /yolo8/runs/detect/train

./                                                train_batch1.jpg
../                                               train_batch2.jpg
args.yaml                                         train_batch4340.jpg
confusion_matrix.png                              train_batch4341.jpg
events.out.tfevents.1744851628.e7a16e1335a7.18.0  train_batch4342.jpg
F1_curve.png                                      val_batch0_labels.jpg
P_curve.png                                       val_batch0_pred.jpg
PR_curve.png                                      val_batch1_labels.jpg
R_curve.png                                       val_batch1_pred.jpg
results.csv                                       val_batch2_labels.jpg
results.png                                       val_batch2_pred.jpg
train_batch0.jpg                                  weights/
